<a href="https://colab.research.google.com/github/Varun1Dt/Sentiment-Analysis/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from wordcloud import WordCloud
import nltk
nltk.download('all',quiet=True)
from PIL import Image

#Model libraries
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df_orignal=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Coronavirus Tweets (1).csv', encoding = 'latin-1')


In [8]:
df1=df_orignal.copy()


In [9]:
df1.head()


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16/03/20,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16/03/20,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16/03/20,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16/03/20,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16/03/20,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [10]:
df1.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [11]:
df=df1[['OriginalTweet','Sentiment']]


In [12]:
df.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


Text Preprocessing
Convert to LowerCase :

In [13]:
df["OriginalTweet"] = df["OriginalTweet"].str.lower()
df['OriginalTweet']

,OriginalTweet
0,@menyrbie @phil_gahan @chrisitv https://t.co/i...
1,advice talk to your neighbours family to excha...
2,coronavirus australia: woolworths to give elde...
3,my food stock is not the only one which is emp...
4,"me, ready to go at supermarket during the #cov..."
...,...
41152,airline pilots offering to stock supermarket s...
41153,response to complaint not provided citing covi...
41154,you know itâs getting tough when @kameronwild...
41155,is it wrong that the smell of hand sanitizer i...


Removing Punctuations :


In [14]:
df['OriginalTweet'] = df['OriginalTweet'].str.replace('http\S+|www.\S+', '', case=False)

In [15]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [16]:
df["clean_tweets"] = df['OriginalTweet'].apply(remove_punctuations)
df['clean_tweets']

,clean_tweets
0,menyrbie philgahan chrisitv httpstcoifz9fan2pa...
1,advice talk to your neighbours family to excha...
2,coronavirus australia woolworths to give elder...
3,my food stock is not the only one which is emp...
4,me ready to go at supermarket during the covid...
...,...
41152,airline pilots offering to stock supermarket s...
41153,response to complaint not provided citing covi...
41154,you know itâs getting tough when kameronwilds...
41155,is it wrong that the smell of hand sanitizer i...


In [17]:
df['clean_tweets'] = df['clean_tweets'].str.replace("[^a-zA-Z#//]"," ")
df.head()

,OriginalTweet,Sentiment,clean_tweets
0,@menyrbie @phil_gahan @chrisitv https://t.co/i...,Neutral,menyrbie philgahan chrisitv httpstcoifz9fan2pa...
1,advice talk to your neighbours family to excha...,Positive,advice talk to your neighbours family to excha...
2,coronavirus australia: woolworths to give elde...,Positive,coronavirus australia woolworths to give elder...
3,my food stock is not the only one which is emp...,Positive,my food stock is not the only one which is emp...
4,"me, ready to go at supermarket during the #cov...",Extremely Negative,me ready to go at supermarket during the covid...


In [18]:
df['clean_tweets'][0]


'menyrbie philgahan chrisitv httpstcoifz9fan2pa and httpstcoxx6ghgfzcc and httpstcoi2nlzdxno8'

Removing Stop Words :

In [19]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [20]:
def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return (text)

In [21]:
df['clean_tweets']= df['clean_tweets'].apply(lambda x: remove_stopwords(x))


**Stemming** is a text preprocessing technique used in Natural Language Processing (NLP) to reduce a word to its base or root form. This helps in normalizing words with the same meaning but different forms. For example, the words "playing," "played," and "plays" can all be reduced to their root form "play.

In [22]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [23]:
def stemming(text):
    text = [stemmer.stem(word) for word in text]
    return (" ".join(text))

In [24]:
df['stemmed'] = df['clean_tweets'].apply(lambda x: stemming(x))


In [25]:
df.stemmed.head()

,stemmed
0,menyrbi philgahan chrisitv httpstcoifz9fan2pa ...
1,advic talk neighbour famili exchang phone numb...
2,coronaviru australia woolworth give elderli di...
3,food stock one empti pleas dont panic enough f...
4,readi go supermarket covid19 outbreak im paran...


Lemmatizing
**Lemmatization** is the process of reducing a word to its base or dictionary form (lemma) while considering its context and part of speech, ensuring the result is a valid word.


In [26]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
df['lemmed'] = df['clean_tweets'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])


In [27]:
df.head()

,OriginalTweet,Sentiment,clean_tweets,stemmed,lemmed
0,@menyrbie @phil_gahan @chrisitv https://t.co/i...,Neutral,"[menyrbie, philgahan, chrisitv, httpstcoifz9fa...",menyrbi philgahan chrisitv httpstcoifz9fan2pa ...,"[menyrbie, philgahan, chrisitv, httpstcoifz9fa..."
1,advice talk to your neighbours family to excha...,Positive,"[advice, talk, neighbours, family, exchange, p...",advic talk neighbour famili exchang phone numb...,"[advice, talk, neighbour, family, exchange, ph..."
2,coronavirus australia: woolworths to give elde...,Positive,"[coronavirus, australia, woolworths, give, eld...",coronaviru australia woolworth give elderli di...,"[coronavirus, australia, woolworth, give, elde..."
3,my food stock is not the only one which is emp...,Positive,"[food, stock, one, empty, please, dont, panic,...",food stock one empti pleas dont panic enough f...,"[food, stock, one, empty, please, dont, panic,..."
4,"me, ready to go at supermarket during the #cov...",Extremely Negative,"[ready, go, supermarket, covid19, outbreak, im...",readi go supermarket covid19 outbreak im paran...,"[ready, go, supermarket, covid19, outbreak, im..."


Tweet Count For Each Sentiment


In [28]:
sentiment_count = df['Sentiment'].value_counts().reset_index()
sentiment_count.columns = ['Sentiment','count']
sentiment_count

,Sentiment,count
0,Positive,11422
1,Negative,9917
2,Neutral,7713
3,Extremely Positive,6624
4,Extremely Negative,5481


In [29]:
# replacing values
replace_values = {"Sentiment":{'Extremely Negative':'Negative', 'Extremely Positive':'Positive'}}
df = df.replace(replace_values)
sentiment_count1 = df['Sentiment'].value_counts().reset_index()
sentiment_count1.columns = ['Sentiment','count']
sentiment_count1

,Sentiment,count
0,Positive,18046
1,Negative,15398
2,Neutral,7713


Train-Test Split

In [30]:
X= df['lemmed']
y=df['Sentiment']
# Applying Train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=10)


In [31]:
print(X_train.head())
y_train.head()


23805    [target, pull, back, store, remodels, opening,...
31828    [breaking, msf, call, patent, profiteering, dr...
24504    [coronavirus, spread, world, leader, get, toug...
17281    [store, shelf, empty, people, selling, certain...
11488    [frescos4u, normal, night, final, reduction, n...
Name: lemmed, dtype: object


,Sentiment
23805,Neutral
31828,Negative
24504,Negative
17281,Positive
11488,Neutral


Count Vectorization (Bag of words)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# Bag of words
cv=CountVectorizer(binary=False,max_df=1.0,min_df=5,ngram_range=(1,2))
cv_X_train=cv.fit_transform(X_train.astype(str).str.strip())

In [35]:
tv=TfidfVectorizer(use_idf=True,max_df=1.0,min_df=5,ngram_range=(1,2),sublinear_tf=True)
tv_X_train=tv.fit_transform(X_train.astype(str).str.strip())


In [36]:
cv_X_test=cv.transform(X_test.astype(str).str.strip())
tv_X_test=tv.transform(X_test.astype(str).str.strip())

Decision Tree Classifier with CV

In [37]:
# Initializing model
dt_cv=DecisionTreeClassifier()

#fitting the data to model
dt_cv.fit(cv_X_train,y_train)

#predicted values
pred_dt_cv=dt_cv.predict(cv_X_test)

In [38]:

pred_dt_cv

array(['Positive', 'Positive', 'Positive', ..., 'Neutral', 'Negative',
       'Positive'], dtype=object)

In [39]:
# Accuracy
cv_score_dt_cv= cross_val_score(dt_cv,cv_X_train,y_train, cv=5)
print("Accuracy: {}" .format(np.mean(cv_score_dt_cv)))

Accuracy: 0.686556060057692


In [40]:
label=['Neutral','Positive','Negative']
print(classification_report(y_test,pred_dt_cv))

              precision    recall  f1-score   support

    Negative       0.69      0.67      0.68      4620
     Neutral       0.61      0.66      0.63      2314
    Positive       0.74      0.74      0.74      5414

    accuracy                           0.70     12348
   macro avg       0.68      0.69      0.69     12348
weighted avg       0.70      0.70      0.70     12348



This model gives an accuracy score of 69.04%
